In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"


In [ ]:
def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")


train_test_split('/content/drive/MyDrive/yolov8/data/')


------ PROCESS STARTED -------
--- This folder has a total number of 608 images---


  0%|          | 0/487 [00:00<?, ?it/s]

------ Training data created with 80% split 487 images -------


  0%|          | 0/121 [00:00<?, ?it/s]

------ Testing data created with a total of 121 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/78.2 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=100 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=football


Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=football3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football3/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images


Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

image 1/4 /content/drive/MyDrive/yolov8/test_images/p.jpg: 448x640 2 potholes, 39.6ms
image 2/4 /content/drive/MyDrive/yolov8/test_images/p1.jpg: 480x640 1 pothole, 61.3ms
image 3/4 /content/drive/MyDrive/yolov8/test_images/potholeimg.jpg: 448x640 1 pothole, 14.0ms
image 4/4 /content/drive/MyDrive/yolov8/test_images/w.jpg: 448x640 (no detections), 13.0ms
Speed: 3.8ms preprocess, 32.0ms inference, 138.2ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8pothole/training_results/football3/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8pothole/vid2


Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 766.0ms
video 1/1 (frame 2/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 644.4ms
video 1/1 (frame 3/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 635.9ms
video 1/1 (frame 4/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 391.6ms
video 1/1 (frame 5/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 385.8ms
video 1/1 (frame 6/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 (no detections), 386.2ms
video 1/1 (frame 7/692) /content/drive/MyDrive/yolov8pothole/vid2/pothole.mp4: 384x640 1 pothole, 378.2ms
video 1/1 (frame 8/692) /content/drive/MyDrive/yolov8p

In [ ]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8pothole/output
